In [3]:
import pandas as pd
import re

In [4]:
file1 = 'data/Energy-Data-Edited.csv'
file2 = 'worldWealthData.csv'

In [5]:
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

In [6]:
df2.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1960 [YR1960]', '1961 [YR1961]', '1962 [YR1962]', '1963 [YR1963]',
       '1964 [YR1964]', '1965 [YR1965]', '1966 [YR1966]', '1967 [YR1967]',
       '1968 [YR1968]', '1969 [YR1969]', '1970 [YR1970]', '1971 [YR1971]',
       '1972 [YR1972]', '1973 [YR1973]', '1974 [YR1974]', '1975 [YR1975]',
       '1976 [YR1976]', '1977 [YR1977]', '1978 [YR1978]', '1979 [YR1979]',
       '1980 [YR1980]', '1981 [YR1981]', '1982 [YR1982]', '1983 [YR1983]',
       '1984 [YR1984]', '1985 [YR1985]', '1986 [YR1986]', '1987 [YR1987]',
       '1988 [YR1988]', '1989 [YR1989]', '1990 [YR1990]', '1991 [YR1991]',
       '1992 [YR1992]', '1993 [YR1993]', '1994 [YR1994]', '1995 [YR1995]',
       '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]', '1999 [YR1999]',
       '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]',
       '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]', '2007 [YR2007]',
       '2008 [YR2008]', '2009 [

### Keeping same countries in both datasets

In [7]:
l1 = set(df1['Country'])
l2 = set(df2['Country Name'])

In [8]:
len(l1), len(l2)

(140, 269)

In [9]:
match = list(l1.intersection(l2))
len(match)

90

In [10]:
not_match = list(l1.difference(l2))
len(not_match)

50

In [11]:
# only keep rows with countries that are in both datasets
df1 = df1[df1['Country'].isin(match)]
df2 = df2[df2['Country Name'].isin(match)]

In [12]:
# rename Country Name column to Country
df2 = df2.rename(columns={'Country Name': 'Country'})

### Renaming year columns for worldHealthData

In [13]:
years_to_be_renamed = [col for col in df2.columns if re.search(r'\d{4}', col)]
for col in years_to_be_renamed:
    year = re.search(r'\d{4}', col).group()
    df2 = df2.rename(columns={col: year})

In [14]:
df2.columns

Index(['Country', 'Country Code', 'Series Name', 'Series Code', '1960', '1961',
       '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970',
       '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',
       '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')

### Cleaning df2

In [15]:
# replace ".." in rows with 0
df2 = df2.replace('..', 0)

### Transforming df2: Cols to rows

In [16]:
min_year = 1980
max_year = 2022

In [17]:
years = [i for i in range(min_year, max_year + 1)]
countries = match

In [18]:
len(years) * len(countries)

3870

In [36]:
# df2_cols = ["Adjusted net national income (annual % growth)", "CO2 emissions from solid fuel consumption (% of total)"]
df2_cols = list(set(df2['Series Name']))
new_cols = ['Country', 'Year']
new_cols.extend(df2_cols)

In [37]:
df3 = pd.DataFrame(columns=new_cols)

# basically convert years that are in columns in df2 to rows
for country in countries:
    for year in years:
        row = [country, year]
        for col in df2_cols:
            row.append(df2[(df2['Country'] == country) & (df2['Series Name'] == col)][str(year)].values[0])
        df3 = df3._append(pd.Series(row, index=new_cols), ignore_index=True)


### Taking a consistent year range

In [38]:
# convert year column to int64
df1['Year'] = df1['Year'].astype('int64')
df3['Year'] = df3['Year'].astype('int64')

In [39]:
# keep only rows with years between min_year and max_year
df1 = df1[(df1['Year'] >= min_year) & (df1['Year'] <= max_year)]
df3 = df3[(df3['Year'] >= min_year) & (df3['Year'] <= max_year)]

In [40]:
len((df1['Country'])), len((df3['Country']))
# len(set(df1['Country'])), len(set(df3['Country']))

(3563, 3870)

In [41]:
countriesToRem = set()
for country in countries:
  # print number of rows that have country in df1 & df3
  if len(df1[df1['Country'] == country]) != len(df3[df3['Country'] == country]):
    print(country, len(df1[df1['Country'] == country]), len(df3[df3['Country'] == country]))
    countriesToRem.add(country)

Papua New Guinea 14 43
New Caledonia 28 43
Madagascar 28 43
Slovenia 33 43
North Macedonia 33 43
Zimbabwe 42 43
Equatorial Guinea 31 43
Chad 21 43
Estonia 38 43
Croatia 33 43
Curacao 11 43
South Sudan 11 43
Zambia 28 43
Serbia 16 43
Mozambique 11 43
Latvia 38 43
Lithuania 38 43
Sudan 29 43
Cuba 28 43
Mongolia 42 43


In [42]:
countries = list(set(countries).difference(countriesToRem))

In [43]:
# keep only rows with countries that are in both datasets
df1 = df1[df1['Country'].isin(countries)]
df3 = df3[df3['Country'].isin(countries)]

In [44]:
len(df1), len(df3)

(3010, 3010)

### Saving datasets

In [45]:
outFile1 = 'edited-data/Energy-Data-Edited.csv'
outFile2 = 'edited-data/worldWealthData-Edited.csv'

In [ ]:
df1.to_csv(outFile1, index=False)
df3.to_csv(outFile2, index=False)